In [11]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import pandas as pd

In [2]:
# Load Train dataset
data = np.loadtxt('train.in')
# helpfulness | 7| 8| 9| 10| 11| 12-31| 32| 33| 34
help_data_X = data[:, 7:33]
help_data_y = data[:, 34]

# clarity | 3| 4| 5| 6| 9| 10| 11| 12-31|32 |33| 35
clar_data_X1 = data[:, 3:7]
clar_data_X2 = data[:, 9:33]
clar_data_X = np.concatenate((clar_data_X1, clar_data_X2), axis=1)
clar_data_y = data[:, 35]

quality = data[:, 36]

In [3]:
hX_train, hX_test, hy_train, hy_test = train_test_split(help_data_X, help_data_y, test_size=0.2, random_state=42)

In [4]:
cX_train, cX_test, cy_train, cy_test = train_test_split(clar_data_X, clar_data_y, test_size=0.2, random_state=42)

In [16]:
# svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
#                    param_grid={"C": [1e0, 1e1, 1e2, 1e3],
#                                "gamma": np.logspace(-2, 2, 5)})

# svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
#                    param_grid={"C": [1e0],
#                                "gamma": np.logspace(-2,5)})
# svr.fit(hX_train, hy_train)
# y_svr = svr.predict(hX_test)
lr = LinearRegression()
# lr = AdaBoostClassifier(n_estimators=400,learning_rate=0.5)
# lr = RandomForestClassifier(n_estimators = 200)
# lr = xgb.XGBClassifier(n_estimators=400,max_depth=5)
lr.fit(hX_train, hy_train)
y_lr = lr.predict(hX_test)
mse = mean_squared_error(hy_test, y_lr)
print("MSE HELP: {}".format(mse))

MSE HELP: 1.85811597446


In [17]:
# svr_c = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
#                    param_grid={"C": [1e0, 1e1, 1e2, 1e3],
#                                "gamma": np.logspace(-2, 2, 5)})
# svr_c.fit(cX_train, cy_train)
# y_svr_c = svr_c.predict(cX_test)
# mse = mean_squared_error(cy_test, y_svr_c)
# print("MSE: {}".format(mse))
clr = LinearRegression()
# clr = AdaBoostClassifier(n_estimators=500,learning_rate=0.3)
# lr = RandomForestClassifier(n_estimators = 500, max_depth = 5)
# clr = xgb.XGBClassifier(n_estimators=500,max_depth=5)
clr.fit(cX_train, cy_train)
y_clr = clr.predict(cX_test)
mse = mean_squared_error(cy_test, y_clr)
print("MSE CLAR: {}".format(mse))

MSE CLAR: 1.70938628591


In [18]:
# y_quality = [0 for i in range(len(y_clr))]
# for i in range(len(y_clr)):
#     y_quality[i] = y_clr[i] + y_lr[i]
# mse = mean_squared_error(quality, y_quality)
# print("MSE ALL: {}".format(mse))
    


In [19]:
# Load Test dataset
data = np.loadtxt('test.in')
# helpfulness | 7| 8| 9| 10| 11| 12-31| 32| 33| 34
help_test = data[:, 7:33]

# clarity | 3| 4| 5| 6| 9| 10| 11| 12-31|32 |33| 35
clar_test1 = data[:, 3:7]
clar_test2 = data[:, 9:33]
clar_test = np.concatenate((clar_test1, clar_test2), axis=1)

In [20]:
lr = LinearRegression()
# lr = AdaBoostClassifier(n_estimators=400,learning_rate=0.5)
# lr = xgb.XGBClassifier(n_estimators=400,max_depth=5)
# lr = RandomForestClassifier(n_estimators = 600, max_depth = 6)
lr.fit(help_data_X, help_data_y)
y_lr = lr.predict(help_test)
clr = LinearRegression()
# clr = AdaBoostClassifier(n_estimators=500,learning_rate=0.3)
# clr = RandomForestClassifier(n_estimators = 600, max_depth = 6)
clr.fit(clar_data_X, clar_data_y)
y_clr = clr.predict(clar_test)

In [12]:
# Read in the data with pandas http://pandas.pydata.org
data = pd.read_csv('train.csv')
data['comments'] = data['comments'].fillna('')
train, test = train_test_split(data)
# Make a pipeline to do unigrams then run linear regression
# http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipeline = Pipeline([
        ('cv', CountVectorizer()),
        ('regression', LinearRegression())
])
cv = GridSearchCV(
    pipeline, {}
).fit(train['comments'], train['quality'])
mse = mean_squared_error(test['quality'], cv.predict(test['comments']))
print("MSE: {}".format(mse))
kaggle_test = pd.read_csv('test.csv')
kaggle_test['comments'] = kaggle_test['comments'].fillna('')
predictions = cv.predict(kaggle_test['comments'])

MSE: 5.07339561145


In [21]:
y_quality = [0 for i in range(len(y_clr))]
outfile = open('predit.out','wb')
for i in range(len(y_clr)):
    y_quality[i] = (y_clr[i] + y_lr[i] + predictions[i]) / 2.0
    outfile.write(str(y_quality[i]) + '\n')
outfile.close()

In [22]:
y_clr[1]

3.5469547592537811